In [ ]:
%run ../helpers/env_variables.py

In [ ]:
import logging
import json
from helpers import handler as h

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch

In [ ]:
## Load configuration
con_file = open(h.scopus_config_file)
config = json.load(con_file)
con_file.close()

In [ ]:
## Initialize client
client = ElsClient(config['apikey'])

In [ ]:
def load_documents(client, my_doc):
    ## Read all documents for example author, then write to disk
    if my_doc.read_docs(client):
        print ("my_auth.doc_list has " + str(len(my_doc.doc_list)) + " items.")
        my_doc.write_docs()
    else:
        print ("Read docs failed.")
    return my_doc

In [ ]:
import helpers.queries as q

In [ ]:
doc_srch = ElsSearch(q.mergoni_query_scopus, 'scopus')
doc_srch.execute(client, get_all = True)
print ("doc_srch has", len(doc_srch.results), "results.")

In [ ]:
mergoni_query = 'ALL(“data envelopment analysis”) AND \
    ALL(“policy evaluation”) \
        AND PUBYEAR > 1956 \
            AND PUBYEAR < 2022 AND \
                LANGUAGE(english)'

In [ ]:
mergoni_query_step_1 = 'ALL(“data envelopment analysis”) AND \
    ALL(“policy evaluation”) \
        AND PUBYEAR > 1956 \
            AND PUBYEAR < 2022'

In [ ]:
mergoni_query_step_1 = 'ALL(“data envelopment analysis”) AND \
    ALL(“policy evaluation”) \
        AND PUBYEAR > 1956 \
            AND PUBYEAR < 2022'

In [ ]:
## Initialize doc search object using Scopus and execute search, retrieving 
#   all results
doc_srch = ElsSearch('ALL(“data envelopment analysis”) AND ALL(“policy evaluation”) AND PUBYEAR > 1956 AND PUBYEAR < 2022','scopus')
doc_srch.execute(client, get_all = True)
print ("doc_srch has", len(doc_srch.results), "results.")